In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd
import os
import cv2

IMAGE_HEIGHT, IMAGE_WIDTH = 448, 448
SEQUENCE_LENGTH = 16

def extract_frames(video_path, output_directory):
    # Video dosyasını oku
    video_reader = cv2.VideoCapture(video_path)

    # Video süresini ve FPS değerini al
    video_duration = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video_reader.get(cv2.CAP_PROP_FPS)

    # Ayırılacak frame aralığı (örneğin 20 saniyede bir frame alalım)
    frame_interval = int(20 * fps)

    # Video adını al (uzantısı olmadan)
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    # Çıkarılan framelerin kaydedileceği klasörü oluştur
    os.makedirs(output_directory, exist_ok=True)

    # Video karelerini çıkart ve kaydet
    frame_counter = 0
    while True:
        # Belirli bir kare pozisyonuna git
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter)

        # Kareyi oku
        success, frame = video_reader.read()

        # Okuma başarısızsa döngüyü sonlandır
        if not success:
            break

        # Belirli aralıklarla frame'i kaydet
        if frame_counter % frame_interval == 0:
            # Yeniden boyutlandırma ve normalizasyon işlemlerini yap
            resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
            normalized_frame = resized_frame / 255

            # Dosyaya yaz
            output_path = os.path.join(output_directory, f"{video_name}_frame_{frame_counter}.jpg")
            cv2.imwrite(output_path, normalized_frame * 255)  # 255 ile çarpılarak normalleştirilmiş piksel değerleri tekrar orijinal değer aralığına getirilir

        # Sonraki frame'e geç
        frame_counter += 1

    # Video okuyucuyu serbest bırak
    video_reader.release()

# Ana klasördeki videoları al
input_directory = "/kaggle/input/youtube-video-download"

# Çıkarılan framelerin kaydedileceği klasör
output_directory = "/kaggle/working/frames"

# İşlenecek video sayısı
max_videos = 2
video_count = 0

# Ana klasördeki videoları işle
for filename in os.listdir(input_directory):
    if filename.endswith(".mp4"):
        # Video dosyasının tam yolunu al
        video_path = os.path.join(input_directory, filename)

        # Framelerin kaydedileceği klasör adı
        video_output_directory = os.path.join(output_directory, os.path.splitext(filename)[0])

        # Frameleri çıkar ve kaydet
        extract_frames(video_path, video_output_directory)
        
        # İşlenen video sayısını artır
        video_count += 1
        
        # İşlenen video sayısı maksimuma ulaştıysa döngüyü sonlandır
        if video_count >= max_videos:
            break
